In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
# http://vrl.cs.brown.edu/color

color = {
    "AS": [82, 239, 153],
    "VT": [17, 94, 65],
    "VF": [148, 210, 207],
    "PE": [44, 69, 125],
    "PR": [209, 121, 248],
    "as": [153, 28, 100],
    "vf": [172, 130, 180],
    "vt": [102, 51, 180],
    "pe": [238, 200, 241],
    "pr": [41, 140, 192],
    "CAS": [183, 209, 101],
    "CVT": [58, 166, 9],
    "CVF": [44, 245, 43],
    "CPE": [93, 64, 48],
    "CPR": [254, 183, 134],
    "cas": [177, 75, 50],
    "cvt": [251, 45, 76],
    "cvf": [251, 189, 19],
    "cpe": [116, 141, 19],
    "cpr": [37, 128, 254],
    "un": [246, 18, 168],
    "cun": [194, 24, 241],
    "dfb": [160, 127, 61],
}

In [18]:
import rerun as rr

rr.init("vivivivivivi")
rr.connect()

In [19]:
import os
from pathlib import Path

import polars as pl
import scipy

from cci.utils import project_dir

oocha_dir = Path(os.environ["OOCHA_DIR"])
clean_df = pl.read_csv(project_dir() / "data/original.csv")
df = clean_df.select(["files", "EPI", "EPI_1", "SMP_start", "SMP_stop", "SMP_stop_1"])
file, epi, epi_1, start, transition, stop = df.row(0)

signal = scipy.io.loadmat(
    oocha_dir / f"{file}.mat",
    simplify_cells=True,
)["SIGNALS"]["ecg_diff"]

In [20]:
signal_2 = scipy.io.loadmat(
    oocha_dir / f"{file}.mat",
    simplify_cells=True,
)["SIGNALS"]
for k, v in signal_2.items():
    print(len(v))
signal_2

372500
372500
372500
372500
372500
372500
372500
372500
372500
372500
372500
372500
372500
372500


{'timecode': array([  8678,   8679,   8680, ..., 381175, 381176, 381177], dtype=int32),
 'pad_switch': array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
 'thorax_imp': array([356.17729, 356.17729, 356.17729, ..., 125.41806, 125.51589,
        125.51589]),
 'ecg_common': array([234.346, 234.308, 234.346, ..., -79.116, -79.078, -79.116]),
 'ecg_offset': array([637.524, 637.416, 637.524, ...,  11.52 ,  11.52 ,  11.52 ]),
 'respiration': array([-23.39424, -23.39568, -23.39496, ...,   4.6008 ,   4.6008 ,
          4.60152]),
 'flow': array([2.375, 2.25 , 2.5  , ..., 5.75 , 5.375, 5.375]),
 'pad_h_acc': array([-2744, -2742, -2741, ..., -2110, -2109, -2107], dtype=int16),
 'ecg_diff': array([-0.071139, -0.070108, -0.076294, ..., -0.058767, -0.057736,
        -0.055674]),
 'board_v_acc': array([-4801, -4801, -4800, ..., -4803, -4796, -4791], dtype=int16),
 'board_h_acc': array([-815, -814, -815, ..., -696, -696, -700], dtype=int16),
 'pad_v_acc': array([-6947, -6947, -6950, ..., -6320, -6319, -6321

In [21]:
tick = 1

for i, (epi, start, stop) in enumerate(
    df.filter(pl.col("files") == file).select(["EPI", "SMP_start", "SMP_stop"]).rows()
):
    rr.log(f"ecg/{epi}/{i}", rr.SeriesLine(color=color[epi], name=epi), timeless=True)
    for value in signal[start:stop]:
        rr.set_time_sequence("step", tick)
        rr.log(f"ecg/{epi}/{i}", rr.Scalar(value))  # , rr.SeriesLine(name=epi, color=color[epi]))
        tick += 1


# rec.show(width=1440, height=1080)